# Onde eu parei mesmo?

Já passou pela situação de começar a ler um livro e esquecer de alguma parte da história ou de um personagem? Ficou com medo de pesquisar no Google sobre o livro e tomar algum spoiler? Vamos resolver este problema!

Este notebook utiliza a API do Google Gemini para criar um chatbot capaz de conversar com você sobre um livro. Ele fornece um resumo  até o capítulo que você leu e conversa sobre os eventos e personagens que apareceram até aquele ponto. Sem spoilers!

## Pré-requisitos

Antes de começar, você vai precisar de duas coisas:

- **API Key do Google Gemini:** você pode criar a sua [neste link](https://aistudio.google.com/app/apikey);

- **PDF do livro:** devido a restrições de direitos autorais, **recomendo utilizar um livro que esteja em domínio público**. No [Portal do Domínio Público](http://www.dominiopublico.gov.br/pesquisa/PesquisaObraForm.jsp) você pode consultar e fazer o download de obras literárias em domínio público ou que tenham a sua divulgação devidamente autorizada.

## Como rodar o Notebook

Para utilizar o notebook, siga os passos abaixo:

1. Sugiro que rode o notebook no [Google Colab](https://colab.research.google.com/), pois ele facilita o upload do livro e armazenamento seguro da API Key;

2. Configuração da API Key: na barra lateral esquerda do Google Colab, clique no ícone de chave 🔑 e então clique em "Adicionar novo secret". Permita acesso ao notebook, dê o nome de "GENAI_API" e cole sua API Key como valor. Se tiver dificuldade, veja um passo a passo [neste vídeo da Alura](https://youtu.be/iwt4bOIHy7s?si=Pf5Qug8Tde0pK5BF&t=4796).

> Caso não queira usar o secrets, você pode descomentar no código a linha a seguir e colar diretamente sua API Key. Lembre-se que, caso faça isso, **não compartilhe** este notebook com ninguém.

```python
# descomente o código abaixo caso queira colar a key diretamente ao invés de
# usar o "Secrets" do Google Colab
# genai.configure(api_key="COLE SUA API KEY AQUI")
```

3. Upload do Livro: na barra lateral esquerda do Google Colab, clique no ícone de pasta 📁 e faça o upload do PDF. Se tiver dificuldade, veja as [instruções deste artigo da Alura](https://cursos.alura.com.br/forum/topico-upload-de-arquivo-do-google-drive-para-o-google-colab-160041).

4. Caminho do PDF: após o upload, clique com o botão direito no PDF para abrir um menu. Clique na opção "Copiar caminho". Se tiver dificuldade, veja as [instruções deste artigo da Alura](https://cursos.alura.com.br/forum/topico-upload-de-arquivo-do-google-drive-para-o-google-colab-160041). Cole este caminho no código do notebook, na variável `conteudo_do_livro`. Veja o exemplo:

  ```python
  # adicione o caminho para o PDF
  conteudo_do_livro = extract_pdf_pages("COLE O CAMINHO AQUI")
  ```

Após seguir estes passos, você pode executar o código e se divertir 🎉

> **Atenção:** O tempo de processamento pode variar dependendo do tamanho do livro e da complexidade da solicitação.

## Divirta-se!

In [ ]:
"""
Código inicial criado no Google AI Studio: https://aistudio.google.com

Este projeto é resultado da minha participação na Imersão Dev IA, promovida pela
Alura em parceiria com o Google.
"""

######## Instalacao e import das bibliotecas necessarias

# SKD do Gemini API
!pip install -q -U google-generativeai

# Leitor de PDF
!pip install -q -U pymupdf

from pathlib import Path
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import textwrap
import hashlib
import pymupdf
import google.generativeai as genai

######## Funcoes auxiliares

# leitura do arquivo pdf
# codigo gerado pelo Gemini, usando a biblioteca PyMuPDF
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]

  # Add logic to read the PDF and return a list of pages here.
  doc = pymupdf.open(pathname)
  pages = []
  for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text = page.get_text("text")
    pages.append(text)

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)

  return parts

# "beautify" da resposta
# créditos: https://ai.google.dev/gemini-api/docs/get-started/python#import_packages
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

######## Criacao do modelo

# use os "Secrets do Google Colab"
genai.configure(api_key=userdata.get('GENAI_API'))

# descomente o código abaixo caso queira colar a key diretamente ao invés de
# usar o "Secrets" do Google Colab
# genai.configure(api_key="COLE SUA API KEY AQUI")

# Set up do modelo
generation_config = {
  "temperature": 0.2,
  "candidate_count": 1,
}

safety_settings = {
    'HARASSMENT': 'BLOCK_ONLY_HIGH',
    'HATE': 'BLOCK_ONLY_HIGH',
    'SEXUAL': 'BLOCK_ONLY_HIGH',
    'DANGEROUS': 'BLOCK_ONLY_HIGH'
}

system_instruction = '''Você irá ajudar a pessoa a se lembrar da história de um
livro e conversar sobre este livro. A primeira mensagem que você vai receber é o
texto do livro em cada página. Junto com este texto, será informado em qual
capítulo a pessoa se encontra neste livro. Em seguida, a pessoa irá fazer
perguntas a respeito desse livro. Suas respostas deverão se basear somente neste
no texto do livro e no capítulo que a pessoa informou. Você não deve contar
sobre acontecimentos ou personagens do livro caso eles ainda não tenham aparecido
até o capítulo que a pessoa informou. Seguem alguns exemplos:

   \n\nExemplo:

   \n\nPrompt: \"Estou no capítulo 7 do livro\"

   \n\nResposta: \"Segue um resumo da história até o capítulo 7: ...\"

   \n\nPrompt: \"O que aconteceu com o nome_do_personagem_1?\"

   \n\nResposta (deverá se basear no que aconteceu no livro até o capítulo
   informado e não deverá contar sobre acontecimentos que a pessoa ainda não
   leu): \"Até onde você leu, o personagem ...\"

   \n\nPrompt: \"Como o personagem Y conheceu o personagem Z?\"

   \n\nResposta (caso esse acontecimento se passe em um capítulo que a pessoa
   não leu): \"Você ainda não chegou nessa parte da história. Continue lendo
   para saber mais!\
'''

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

######## Rodando a aplicacao em formato de chat

# adicione o caminho para o PDF
conteudo_do_livro = extract_pdf_pages("COLE O CAMINHO AQUI")

chat = model.start_chat(history=[])

# interacao inicial para saber ate onde a pessoa leu
print('Em qual capítulo você parou? Por exemplo: "capítulo 9"')
capitulo_atual = input()
print(f'\n\nQue legal. Aqui está um resumo do livro até agora: \n\n')

prompt_parts = [
  *conteudo_do_livro,
  f"Estou no {capitulo_atual}. Suas respostas devem se basear até este capítulo. Não conte sobre personagens ou acontecimentos que ainda não tenham acontecido até este capítulo.",
  f"Faça um resumo breve até o {capitulo_atual}. No máximo 5 parágrafos."
]
response = chat.send_message(prompt_parts)
display(to_markdown(response.text))

# demais interacoes para conversar sobre o livro
print('\n\nO que quer saber da história até aqui? Digite "sair" para encerrar.')
prompt = input()

while prompt != 'sair':
  response = chat.send_message(prompt)
  display(to_markdown(response.text))
  print('\n\nO que mais quer saber? Digite "sair" para encerrar.')
  prompt = input()

print('\n\nObrigado por conversar comigo. Até a próxima!')